In [13]:
from __future__ import division
from copy import deepcopy as dc
import numpy as np
import matplotlib.pyplot as mpl

In [16]:
def get_M(x,s):  # 获取对应扩散率曲线的最优潜在市场容量
    len_s = len(s)
    p,q = x[:2]
    f = x[2:len_s+2]
    a = np.sum(np.square(f))/np.sum(s)  # 除以np.sum(self.s)是为减少a的大小
    b = -2*np.sum(f*s)/np.sum(s)
    c = np.sum(np.square(s))/np.sum(s)
    mse,sigma = np.sqrt(sum(s)*(4*a*c - b**2)/(4*a*len_s)),-b/(2*a)
    m = sigma*10000
    return mse,m

In [17]:
def fit(db,s):
    f_cont = [get_M(d,s) for d in db]
    idx = np.argmin([x[0] for x in f_cont])
    return {'m':f_cont[idx][1],'p':db[idx][0],'q':db[idx][1],'curve':db[idx][2:]*f_cont[idx][1]/10000}

In [52]:
def combine_predict(s,w_c = 1,db_cont=db_cont):
    len_s = len(s)
    one_step = []
    w_list = [1/len(db_cont) for l in range(len(db_cont))] #初始权重
    for i in range(6,len_s-1):
        one_list = []
        s_list = []
        for db in db_cont:
            res = fit(db,s[:i])
            c = res['curve']
            one = [abs(c[i]-s[i])/s[i],abs(c[i]-s[i]),(c[i]-s[i])**2]#MPE,MAD,MSE
            s_list.append(c[i])
            one_list.append(one)
        
        com_pred = np.sum(np.multiply(s_list,w_list))
        one_list.append([abs(com_pred-s[i])/s[i], abs(com_pred-s[i]), (com_pred-s[i])**2])
        one_step.append(one_list)
        
        trans = [x[1] for x in one_list[:len(db_cont)]] #提取出各ABM模型预测的MAD
        trans_list = [(max(trans)-x)/(max(trans)-min(trans)) for x in trans] #得出相对预测精度
        w_update = [a+w_c*b for a,b in zip(w_list,trans_list)] #先验和后验1:w_c
        w_list = [x/sum(w_update) for x in  w_update]   #学习，更新权重
        
    return one_step

In [6]:
text_set = ['lognorm','ba','exp','gau','er','ws0','ws0.1','ws0.3','ws0.5','ws0.7','ws0.9']
db_cont = [ np.load('diffusion data set for %s.npy'%tx) for tx in text_set]

In [7]:
data_set = {'room air conditioners':(np.arange(1949,1962),[96,195,238,380,1045,1230,1267,1828,1586,1673,1800,1580,1500]),
                        'color televisions':(np.arange(1963,1971),[747,1480,2646,5118,5777,5982,5962,4631]),
                        'clothers dryers':(np.arange(1949,1962),[106,319,492,635,737,890,1397,1523,1294,1240,1425,1260,1236])}

### Room air conditioners

In [56]:
s1 = data_set['room air conditioners'][1]
c_predict1 = combine_predict(s1,w_c = 0)
print np.mean(c_predict1,axis=0)

[[  1.13352105e-01   1.94537836e+02   7.50221705e+04]
 [  1.63238010e-01   2.60686548e+02   9.85581140e+04]
 [  1.78274494e-01   3.07319287e+02   1.60493511e+05]
 [  1.98060272e-01   3.37656791e+02   1.79344899e+05]
 [  1.69278278e-01   2.96612036e+02   1.59017997e+05]
 [  1.67569579e-01   2.73947852e+02   9.36147596e+04]
 [  2.35576753e-01   3.64580151e+02   1.87699251e+05]
 [  2.68165452e-01   4.14825052e+02   2.38177505e+05]
 [  1.95452234e-01   3.37840688e+02   1.81208061e+05]
 [  2.09248253e-01   3.61620288e+02   2.12770445e+05]
 [  2.09175066e-01   3.56473454e+02   1.86853516e+05]
 [  1.71221900e-01   2.86131274e+02   1.23713319e+05]]


### Color televisions

In [57]:
s2 = data_set['color televisions'][1]
c_predict2 = combine_predict(s2,w_c = 1)
np.mean(c_predict2,axis=0)

array([[  1.11850492e-01,   6.66852636e+02,   4.44692438e+05],
       [  6.17109330e-02,   3.67920582e+02,   1.35365555e+05],
       [  1.55619147e-01,   9.27801352e+02,   8.60815348e+05],
       [  2.31135430e-01,   1.37802943e+03,   1.89896512e+06],
       [  2.10398315e-01,   1.25439476e+03,   1.57350621e+06],
       [  3.59224136e-01,   2.14169430e+03,   4.58685446e+06],
       [  4.26851946e-02,   2.54489130e+02,   6.47647173e+04],
       [  1.90680193e-01,   1.13683531e+03,   1.29239453e+06],
       [  2.16132320e-01,   1.28858089e+03,   1.66044071e+06],
       [  1.66610865e-01,   9.93333978e+02,   9.86712391e+05],
       [  1.42997673e-01,   8.52552127e+02,   7.26845129e+05],
       [  9.86569126e-02,   5.88192513e+02,   3.45970432e+05]])

### Clothers dryers

In [58]:
s3 = data_set['clothers dryers'][1]
c_predict3 = combine_predict(s3,w_c = 1)
np.mean(c_predict3,axis=0)

array([[  2.03438828e-01,   2.78470733e+02,   1.13152708e+05],
       [  2.04273921e-01,   2.71112368e+02,   1.20424967e+05],
       [  2.42127460e-01,   3.23682112e+02,   2.38358612e+05],
       [  2.18793843e-01,   2.99880823e+02,   1.18092018e+05],
       [  2.27814163e-01,   3.12057476e+02,   1.33962038e+05],
       [  2.26614292e-01,   3.00307660e+02,   1.32973508e+05],
       [  2.00044417e-01,   2.70237633e+02,   1.27925394e+05],
       [  2.05707913e-01,   2.79242008e+02,   1.26783266e+05],
       [  2.24541444e-01,   3.05306106e+02,   1.18363084e+05],
       [  2.09519604e-01,   2.85638768e+02,   1.22247426e+05],
       [  2.51686343e-01,   3.44620486e+02,   1.57133681e+05],
       [  2.18163971e-01,   2.94521743e+02,   1.35874133e+05]])